## Installing dependencies

In [ ]:
#!pip install -r requirements.txt

# Imports

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import pathlib
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from tqdm import tqdm
import wandb
import torch.nn.functional as F

# Data paths

In [3]:
DATA_ROOT = pathlib.Path(r"data\archive\food-101\food-101")
IMAGE_DIR = pathlib.Path("data/archive/food-101/food-101/images")

#DATA_ROOT = pathlib.Path(r"E:\\food-101\\food-101")
#IMAGE_DIR = pathlib.Path("E:\\food-101\\food-101\\images")

if not IMAGE_DIR.exists():
    raise FileNotFoundError(f"Missing dataset folder: {IMAGE_DIR}")

print("Dataset path OK.")


Dataset path OK.


# Data preprocessing

In [4]:
img_size = 224

train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

full_dataset = datasets.ImageFolder(IMAGE_DIR, transform=train_transform, allow_empty=True)
class_names = full_dataset.classes
num_classes = len(class_names)

print("Total images:", len(full_dataset))
print("Classes:", num_classes)


Total images: 101000
Classes: 101


# Split data into sets

In [5]:
train_size = int(0.7 * len(full_dataset))
val_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(full_dataset, [train_size, val_size, test_size])

# Create Dataloaders

In [6]:
val_ds.dataset.transform = test_transform
test_ds.dataset.transform = test_transform

batch_size = 32

train_loader = DataLoader(
    train_ds,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)


# Create Model

In [7]:
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
model = efficientnet_b0(weights=weights)

# Replace classifier head
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Freeze feature extractor
for param in model.features.parameters():
    param.requires_grad = False

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


Device: cpu


## Initialize a W&B

In [ ]:
# W&B init
wandb.login()  # will prompt you the first time in this environment

wandb_config = {
    "model_name": "efficientnet_b0",
    "img_size": img_size,
    "batch_size": 32,
    "optimizer": "Adam",
    "learning_rate": 1e-3,
    "train_size": len(train_ds),
    "val_size": len(val_ds),
    "test_size": len(test_ds),
    "num_classes": num_classes,
}

run = wandb.init(
    project="food101-efficientnet",
    config=wandb_config,
    name="efficientnet_b0_food101_run_1",  # you can change this per run
)

wandb.watch(model, log="all", log_freq=100)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:wandb: Paste an API key from your profile and hit enter:

# Define training and evaluation functions

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device, epoch, log_interval=100):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    pbar = tqdm(loader, desc=f"Training Epoch {epoch}", leave=False)

    for batch_idx, (imgs, labels) in enumerate(pbar):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

        avg_loss_so_far = total_loss / total
        avg_acc_so_far = correct / total

        pbar.set_postfix({
            "loss": f"{avg_loss_so_far:.4f}",
            "acc": f"{100 * avg_acc_so_far:.2f}%"
        })

        # Optional: batch-level logging to W&B
        if (batch_idx + 1) % log_interval == 0:
            wandb.log(
                {
                    "batch/train_loss": loss.item(),
                    "batch/lr": optimizer.param_groups[0]["lr"],
                    "epoch": epoch,
                }
            )

    avg_loss = total_loss / total
    avg_acc = correct / total

    # Epoch-level logging to W&B
    wandb.log(
        {
            "epoch/train_loss": avg_loss,
            "epoch/train_accuracy": avg_acc,
            "epoch/train_error_rate": 1 - avg_acc,
            "epoch": epoch,
        }
    )

    return avg_loss, avg_acc

In [ ]:
def evaluate(model, loader, criterion, device, epoch=None, split="val"):
    """
    Generic evaluation for val/test.
    split: "val" or "test" (used in W&B metric names).
    """
    model.eval()
    total_loss, correct, total = 0.0, 0, 0

    desc = f"Evaluating ({split})"
    if epoch is not None:
        desc += f" Epoch {epoch}"

    with torch.no_grad():
        pbar = tqdm(loader, desc=desc, leave=False)

        for imgs, labels in pbar:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * imgs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

            avg_loss_so_far = total_loss / total
            avg_acc_so_far = correct / total

            pbar.set_postfix({
                "loss": f"{avg_loss_so_far:.4f}",
                "acc": f"{100 * avg_acc_so_far:.2f}%"
            })

    avg_loss = total_loss / total
    avg_acc = correct / total

    # Log to W&B if epoch is known (for val) or just once (for test)
    metric_prefix = f"{split}"
    log_data = {
        f"{metric_prefix}/loss": avg_loss,
        f"{metric_prefix}/accuracy": avg_acc,
        f"{metric_prefix}/error_rate": 1 - avg_acc,
    }
    if epoch is not None:
        log_data["epoch"] = epoch

    wandb.log(log_data)

    return avg_loss, avg_acc

# Train model

In [ ]:
epochs = 5

for epoch in range(1, epochs + 1):
    # ---- Debug: hardware info ----
    print(f"\n--- Epoch {epoch}/{epochs} Diagnostics ---")
    print("Using device:", device)

    # Check model parameters
    first_param = next(model.parameters())
    print("Model param device:", first_param.device)

    if device == "cuda":
        print("Allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")
        print("Reserved:", torch.cuda.memory_reserved() / 1024**2, "MB")

    print("-------------------------------------------\n")

    # ---- Training ----
    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion, device, epoch
    )

    # ---- Validation ----
    val_loss, val_acc = evaluate(
        model, val_loader, criterion, device, epoch=epoch, split="val"
    )

    # ---- Epoch Summary ----
    print(f"Epoch {epoch}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")



--- Epoch 1/5 Diagnostics ---
Using device: cuda
Model param device: cuda:0
Allocated: 87.54541015625 MB
Reserved: 662.0 MB
-------------------------------------------



Epoch 1/5
  Train Loss: 2.5606, Acc: 0.4107
  Val   Loss: 2.0274, Acc: 0.5018

--- Epoch 2/5 Diagnostics ---
Using device: cuda
Model param device: cuda:0
Allocated: 89.0263671875 MB
Reserved: 668.0 MB
-------------------------------------------



Epoch 2/5
  Train Loss: 2.0194, Acc: 0.5044
  Val   Loss: 1.9290, Acc: 0.5244

--- Epoch 3/5 Diagnostics ---
Using device: cuda
Model param device: cuda:0
Allocated: 89.0263671875 MB
Reserved: 668.0 MB
-------------------------------------------



Epoch 3/5
  Train Loss: 1.9219, Acc: 0.5213
  Val   Loss: 1.9018, Acc: 0.5263

--- Epoch 4/5 Diagnostics ---
Using device: cuda
Model param device: cuda:0
Allocated: 89.0263671875 MB
Reserved: 668.0 MB
-------------------------------------------



Epoch 4/5
  Train Loss: 1.8626, Acc: 0.5354
  Val   Loss: 1.8853, Acc: 0.5320

--- Epoch 5/5 Diagnostics ---
Using device: cuda
Model param device: cuda:0
Allocated: 89.0263671875 MB
Reserved: 668.0 MB
-------------------------------------------



Epoch 5/5
  Train Loss: 1.8307, Acc: 0.5406
  Val   Loss: 1.8868, Acc: 0.5330


# Test model

In [ ]:
test_loss, test_acc = evaluate(
    model, test_loader, criterion, device, epoch=None, split="test"
)

print("\n=== Test Results ===")
print(f"Test Loss:     {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print("====================\n")


=== Test Results ===
Test Loss:     1.8994
Test Accuracy: 0.5361



In [ ]:
wandb.finish()